# Simulating Process Flow

# Stages of Manufacturing

## 1. [Raw Material Inventory Store](#first-bullet)

## 2. [Classifier](#second-bullet)

## 3. [Pre-Finish Inventory Store](#third-bullet)

## 4. [Pre-Finish Operation](#fourth-bullet)

## 5. [Pack Inventory Store](#fifth-bullet)

## 6. [Packaging](#sixth-bullet)

![process_flow_diagram](process_flow_diagram.svg)

## 1. Raw Material Inventory (RMI) Store <a class="anchor" id="first-bullet"></a>
Several drums act as an inventory store for raw material. The number of drums and their capacities varies by site. Drums can only hold jelly beans with the same color (many-to-one relationship between drums and jelly beans of a given color). One RMI drum can be released at a time into the classifier.

Rules:
1. Order of release from RMI drums is dictated by workorders
2. If there are two RMI drums with the same color at a given location, start emptying the drum with the lower equipment number first
3. Releasing jelly beans from a drum is only allowed if there are at least 5 pre-finished inventory bins empty and available (the next step, classification, is a continuous process which fills pre-finished inventory bins)

In [1]:
import pandas as pd
import numpy as np
# from sklearn import 

In [2]:
class facility:
    def __init__(self, location):
        self.location = location
        print("Building {0} Facility Details...".format(location))
        
        # TODO - utilize equipment metadata from file
        equipment = pd.read_csv('equipment.csv')
        
        self.rmi_store = rmi_store(location)
        self.classifier = classifier(location)
        self.pfi_store = pfi_store(location)
        
    def __repr__(self):
        return("{0}".format(self.location))

In [6]:
class rmi_store:
    def __init__(self, location):
        # Initialize RMI Store for a given location w/ general properties
        rmi_il = pd.read_csv('rmi_inventory_level.csv')
        rmi_il = rmi_il[rmi_il['Location Name'] == location]

        
        self.facility = location
        self.num_drums = len(rmi_il)
        self.colors = rmi_il['Color'].dropna()
        
        self.fill_store(rmi_il)
        #self.order_drums()
        
    def __repr__(self):
        return(
            "{0} RMI Store \n"
            "Containing {1} Drums \n"
            "With {2} Unique Colors"
            .format(self.facility, self.num_drums, len(self.colors))
        )        
    
    def fill_store(self, df):
        print('Filling RMI Drums...')
        self.drums = [
            rmi_drum(
            row['RMI Drum'],
            row['Color'],
            row['Qty in pounds'],
            row['Capacity']
            )
            for index, row in df.iterrows()
        ]
    
    def order_drums():
    # Sort list of drums according to work order
        pass
            
            
class rmi_drum:
    def __init__(self, drum_id, color, start_qty, capacity):
        self.id = drum_id
        self.color = color
        self.start_qty = start_qty
        self.capacity = capacity
        #self.fill()
    
    def __repr__(self):
        return(self.id)
    
    def fill(self, addtl_qty):
        self.final_qty = self.start_qty + self.addtl_qty
        
    def empty(self):
        self.lb_qty = 0

In [100]:
detroit = facility('Detroit, MI')
print(detroit.rmi_store)
print(detroit.rmi_store.drums[0:5])
print(detroit.rmi_store.drums[1])
print(detroit.rmi_store.drums[1].capacity)

Building Detroit, MI Facility Details...
Filling RMI Drums...
Initializing Classifier...
Defining PFI Drums...
Detroit, MI RMI Store 
Containing 40 Drums 
With 38 Unique Colors
[RMI DRUM1, RMI DRUM2, RMI DRUM3, RMI DRUM4, RMI DRUM5]
RMI DRUM2
300,000


In [101]:
columbus = facility('Columbus, OH')
print(columbus.rmi_store)
print(columbus.rmi_store.drums[0:5])
print(columbus.rmi_store.drums[1])
print(columbus.rmi_store.drums[1].capacity)

Building Columbus, OH Facility Details...
Filling RMI Drums...
Initializing Classifier...
Defining PFI Drums...
Columbus, OH RMI Store 
Containing 30 Drums 
With 28 Unique Colors
[RMI DRUM1, RMI DRUM2, RMI DRUM3, RMI DRUM4, RMI DRUM5]
RMI DRUM2
320,000


## 2. Classifier <a class="anchor" id="second-bullet"></a>

The RMI drums contain jelly beans of the same color, but of varying size. RMI drums are emptied one by one into the classifier, which sorts jelly beans into 5 distinct sizes. Percentage split of each jelly bean size for a given color is provided with the problem statement.

In [99]:
class classifier:
    def __init__(self, location):
        print('Initializing Classifier...')
        rates = pd.read_csv('classifier_rate.csv')
        
        self.color = 'Undefined'
        self.location = location
        self.rate = int(rates[rates['Site'] == location]['Processing_Rate'])
        self.id = rates[rates['Site'] == location]['Classifier'].values[0]
        
    
    def __repr__(self):
        return("{0} Classifier\n"
               "ID Number: {1}\n"
               "Processing Color: {2}\n"
               "At Rate: {3} lbs/hr"
               .format(self.location,
                       self.id,
                       self.color,
                       self.rate
                      )
              )
    
    def check_pfi_bins(self):
        pass
    
    def classify(self, color, incoming):
        # Return quantity of each size of jelly bean
        print("Classifying {0} {1} Jelly Beans...".format(incoming, color))
        split = pd.read_csv('classifier_split.csv')
        
        self.color = color
        self.split = split[split['Color'] == color]
        
        self.check_pfi_bins()
        split = self.split
        split['qty_out'] = split.apply(lambda x: int(incoming*x.Percentage/100), axis=1)
        time = self.rate/incoming
        print("Classifying Time: {0} Hours".format(time))
        return split[['Size','qty_out']]
        
        
        

In [12]:
detroit.classifier

Detroit, MI Classifier
ID Number: Classifier 1
Processing Color: Undefined
At Rate: 3420 lbs/hr

In [13]:
detroit.classifier.classify('Coloring Agent1', 10000)

Classifying 10000 Coloring Agent1 Jelly Beans...
Classifying Time: 0.342 Hours


,Size,qty_out
0,S1,1200
1,S5,2700
2,S4,1900
3,S2,2900
4,S3,1300


## 3. Pre-Finish Inventory (PFI) Store <a class="anchor" id="third-bullet"></a>

Jelly beans sorted by the classifier are stored in PFI drums. Drums are assigned a specific jelly bean size and color to avoid contamination (the PFI drums are reused once the drums are emptied into the Pre-Finish Operation). As mentioned earlier, 5 PFI drums (one for each size) must be empty before an RMI drum can be unloaded (thus starting the classification process).

Rules:
1. The classification process simultaneously splits the jelly beans according to the given ratios directly into the PFI drums
2. Fundamentally any PFI drum can be filled with any size of jelly bean; however, once the assignment is made for a given work order, the PFI drum must only hand jelly beans of that size & color until the work order is completely processed
3. PFI drums can only be filled to 95% capacity
4. PFI drums are only available to be emptied into the Pre-Finish Operation once filling has stopped

In [9]:
class pfi_store:
    def __init__(self, location):
        pfi_drums = pd.read_csv('pfi_drum.csv', thousands=',')
        pfi_drums = pfi_drums[pfi_drums['Site'] == location]
        
        self.facility = location
        self.num_drums = len(pfi_drums)
        
        self.fill_store(pfi_drums)
        self.assign_drums()
        
    def __repr__(self):
        return(
            "{0} PFI Store \n"
            "Containing {1} Drums"
            .format(self.facility, self.num_drums)
        )     
        
    def fill_store(self, df):
        print('Defining PFI Drums...')
        self.drums = [
            pfi_drum(
            row['Drum Number'],
            row['Capacity In pounds']
            )
            for index, row in df.iterrows()
        ]
        
    def assign_drums(self):
        pass
        
class pfi_drum:
    def __init__(self, drum_id, capacity):
        self.id = drum_id
        self.capacity = int(capacity)
        self.contents = 0
        
    def __repr__(self):
        return(self.id)
    
    def assign(self, jb_color, jb_size):
        self.jb_color = jb_color
        self.jb_size = jb_size
    
    def fill(self, amount):
        if (self.contents + amount) <= 0.95*self.capacity:
            self.contents += amount
        else:
            raise ValueError('Amount specified exceeds drum capacity')
        
    def empty(self):
        self.contents = 0

In [16]:
detroit.pfi_store

Detroit, MI PFI Store 
Containing 15 Drums

In [17]:
print(detroit.pfi_store.drums)
print(detroit.pfi_store.drums[1].capacity)

[PFI Drum1, PFI Drum2, PFI Drum3, PFI Drum4, PFI Drum5, PFI Drum6, PFI Drum7, PFI Drum8, PFI Drum9, PFI Drum10, PFI Drum11, PFI Drum12, PFI Drum13, PFI Drum14, PFI Drum15]
10000


In [21]:
detroit.pfi_store.drums[0].assign(jb_color ='Coloring Agent1', jb_size ='S1')
detroit.pfi_store.drums[0].jb_color

'Coloring Agent1'

In [22]:
detroit.pfi_store.drums[0].capacity

10000

In [23]:
detroit.pfi_store.drums[0].fill(9000)
detroit.pfi_store.drums[0].contents

9000

In [24]:
detroit.pfi_store.drums[0].fill(501)
detroit.pfi_store.drums[0].contents

ValueError: Amount specified exceeds drum capacity

In [25]:
detroit.pfi_store.drums[0].empty()
detroit.pfi_store.drums[0].contents

0

## 4. Pre-Finish Operation (PFO) <a class="anchor" id="fourth-bullet"></a>

PFI drums are now emptied into a tank which applies flavoring to jelly beans of a given color and size (i.e., the tank can only hold 1 unique color-size-flavor combinations at any given time). The number of tanks, tank capacity, and processing rate are given for each facility as part of the problem statement.

Rules:
1. If there is more than 1 PFO tank, assume the rate for each tank is the same.
2. When flavors are changed, there is a change-over duration of 5 minutes.
3. There is no change-over duration between size changes.

In [102]:
class pfo_tank:
    def __init__(self, location):
        self.facility = location
        self.color = "Undefined"
        self.rate = np.nan
        self.flavor = "Undefined"
        
        
    def __repr__(self):
        print("{0} PFO Tank\n"
              "Processing Color: {1}\n"
              "Processing Flavor: {2}\n"
              "Predicted Processing Rate: {3}"
              .format(
                  self.location,
                  self.color,
                  self.flavor,
                  self.rate
              )
             )
    
    def apply(self, flavor):
        self.load(flavor)
        self.flavor = flavor
        pass
    
    def load(self, flavor, color, size):
        if self.flavor != flavor:
            process_time += 5
        else:
            pass
            

SyntaxError: invalid syntax (<ipython-input-102-2e0b5034cd63>, line 28)